## SICP 习题 （3.18）解题总结: 环路的检测

SICP 习题 3.18 要求我们检测一个列表是否包含环路，就是对一个列表不断执行cdr会不会一直循环下去。

关于环路我们在习题3.13里已经讨论过，它可以用下面的(make-cycle)函数创建：

In [30]:
(define (make-cycle x)
  (set-cdr! (last-pair x) x)
  x)


(define (last-pair x)
  (if (null? (cdr x))
      x
      (last-pair (cdr x))))

要检测环路，主要是看遍历过程中是不是总是遇到同一个元素。但是要注意，我们要检查是不是通过同一个路径不断到达这个元素，我们不能简单地认为看见一个之前看过的元素就是形成了环路。

简单的思路就是在行进过程中不断记录自己走过的元素，如果遇到分支的话要分别创建不同分支的路径，不同分支不能共享一个路径数据，不然的话两个分支指向同一个元素也会被误认为是形成环路。

为不同的分支创建不同的路径听起来挺复杂，但是我们可以利用递归调用的变量处理能力自动完成这个工作。

主体代码如下：

In [31]:
(define (loop-detect x track-path)
  (if (not (pair? x))
      #f
      (if (element-of-set? x track-path)
          #t
          (or (loop-detect (car x) (adjoin-set x track-path))
              (loop-detect (cdr x) (adjoin-set x track-path))))))

为了记录和检测元素，我们还是需要之前创建的set的处理代码：

In [32]:
(define (element-of-set? x set)
  (cond ((null? set) #f)
	((eq? x (car set)) #t)
	(else (element-of-set? x (cdr set)))))

(define (adjoin-set x set)
  (if (element-of-set? x set)
      set
      (cons x set)))

先测试一下不是环路但是有共享元素的情况：

In [33]:
(define leaf (cons 1 2))
(define to-same-leaf (cons leaf leaf))
(define test-sample (cons to-same-leaf leaf))
(loop-detect test-sample '(0))

#f

然后用make-cycle创建一个环路样例进行测试：

In [34]:
(define cycle-sample (make-cycle '(1 2 3 4)))
(loop-detect cycle-sample '(0))

#t